In [2]:
import os, sys
sys.path.append("../../common/")
from python_tools import *

python tools loaded.


In [3]:
%matplotlib notebook

In [7]:
TRUTH_DF_FILE = "/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/mcana_dfs.h5"
PELEE_1ENP_FILES = ["/Users/wketchum/Data/MicroBooNE/FakeData2020/PeLEE/fakedata1/data_1eNp_fakedata1.root"]
PELEE_1E0P_FILES = ["/Users/wketchum/Data/MicroBooNE/FakeData2020/PeLEE/1e0p/prod_uboone_nu2020_fakedata_set1_run1_reco2_v08_00_00_41_reco2_fakedata1_1e0p_BDT_sbnfit.root",
                    "/Users/wketchum/Data/MicroBooNE/FakeData2020/PeLEE/1e0p/prod_uboone_nu2020_fakedata_set1_run3b_reco2_v08_00_00_41_reco2_fakedata1_1e0p_BDT_sbnfit.root"]
DLLEE_FILES = ["/Users/wketchum/Data/MicroBooNE/FakeData2020/DLeLEE/sel1e1p_fakedata_set1_run1_eventlist.txt",
               "/Users/wketchum/Data/MicroBooNE/FakeData2020/DLeLEE/sel1e1p_fakedata_set1_run3_eventlist.txt"]

In [54]:
df_n = pd.read_hdf(TRUTH_DF_FILE,"ev_df")
df_n = df_n.reset_index()
df_n = df_n.astype({"run":"int64","subrun":"int64","event":"int64"})

In [55]:
pelee_1eNp_df = pd.concat([uproot.open(f)["NeutrinoSelectionFilter"].pandas.df() for f in PELEE_1ENP_FILES])
pelee_1e0p_df = pd.concat([uproot.open(f)["NeutrinoSelectionFilter"].pandas.df() for f in PELEE_1E0P_FILES])
pelee_1eNp_df = pelee_1eNp_df.rename(columns={"sub":"subrun","evt":"event"})
pelee_1e0p_df = pelee_1e0p_df.rename(columns={"sub":"subrun","evt":"event"})

In [56]:
dl_df = pd.concat([pd.read_csv(f,names=["run","subrun","event","reco_nu_e"],header=None) for f in DLLEE_FILES])

In [62]:
def find_truth(df_t,r,sr,ev):
    return df_t.query("run==%d and subrun==%d and event==%d"%(r,sr,ev))[["run","subrun","event",
                                                                         "n_mu","n_e","n_p_40MeV",
                                                                         "n_pi0","n_gamma"]]

In [64]:
def print_breakdown(df_n,selection_df,selection_label="Selection"):
    n_1eNp = 0
    n_1e0p = 0
    n_1e1p = 0
    n_1gNp = 0
    n_1g1p = 0
    n_1g0p = 0
    n_ncpi0 = 0
    n_ccpi0 = 0
    n_bnbother = 0
    n_notfound = 0
    for i,ev in selection_df.iterrows():
        mct = find_truth(df_n,ev["run"],ev["subrun"],ev["event"])
        if(len(mct)==0):
            print("Error: cannot find run %d, subrun %d, event %d"%(e["run"],e["subrun"],e["event"]))
            n_notfound+=1
            continue;
        mct = mct.iloc[0]
        if(mct["n_e"]>0):
            if(mct["n_p_40MeV"]>0): 
                n_1eNp+=1
                if(mct["n_p_40MeV"]==1): n_1e1p+=1
            elif(mct["n_p_40MeV"]==0): 
                n_1e0p+=1
        elif(mct["n_gamma"]>0):
            if(mct["n_p_40MeV"]>0): 
                n_1gNp+=1
                if(mct["n_p_40MeV"]==1): n_1g1p+=1
            elif(mct["n_p_40MeV"]==0): 
                n_1g0p+=1
        elif(mct["n_pi0"]>0):
            if(mct["n_mu"]==0): n_ncpi0+=1
            if(mct["n_mu"]==1): n_ccpi0+=1
        else:
            n_bnbother+=1
    
    print("%s Total Events: %d"%(selection_label,len(selection_df)))
    print("\t1e0p: %d, 1e1p: %d, 1eNp (incl. 1e1p): %d"%(n_1e0p,n_1e1p,n_1eNp))
    print("\t1g0p: %d, 1g1p: %d, 1gNp (incl. 1g1p): %d"%(n_1g0p,n_1g1p,n_1gNp))
    print("\tNC pi0: %d, CC pi0: %d"%(n_ncpi0,n_ccpi0))
    print("\tOther BNB: %d"%n_bnbother)


In [67]:
print_breakdown(df_n,pelee_1eNp_df,"PeLEE 1eNP Selection")

PeLEE 1eNP Selection Total Events: 67
	1e0p: 0, 1e1p: 48, 1eNp (incl. 1e1p): 56
	1g0p: 0, 1g1p: 0, 1gNp (incl. 1g1p): 1
	NC pi0: 4, CC pi0: 2
	Other BNB: 4


In [69]:
print_breakdown(df_n,pelee_1e0p_df,"PeLEE 1e0P Selection")

PeLEE 1e0P Selection Total Events: 28
	1e0p: 7, 1e1p: 9, 1eNp (incl. 1e1p): 9
	1g0p: 0, 1g1p: 0, 1gNp (incl. 1g1p): 0
	NC pi0: 8, CC pi0: 1
	Other BNB: 3


In [70]:
print_breakdown(df_n,dl_df,"DL LEE 1e1P Selection")

DL LEE 1e1P Selection Total Events: 49
	1e0p: 0, 1e1p: 35, 1eNp (incl. 1e1p): 39
	1g0p: 0, 1g1p: 1, 1gNp (incl. 1g1p): 1
	NC pi0: 5, CC pi0: 2
	Other BNB: 2
